In [1]:
#Reference : https://www.tensorflow.org/tutorials/text/nmt_with_attention

In [60]:
!wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip

--2021-05-08 06:19:03--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6201435 (5.9M) [application/zip]
Saving to: ‘NotoSansDevanagari-hinted.zip’

NotoSansDevanagari- 100%[===================>]   5.91M  37.6MB/s    in 0.2s    

2021-05-08 06:19:04 (37.6 MB/s) - ‘NotoSansDevanagari-hinted.zip’ saved [6201435/6201435]



In [63]:
!unzip /content/NotoSansDevanagari-hinted.zip

Archive:  /content/NotoSansDevanagari-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSansDevanagari-Black.ttf  
  inflating: NotoSansDevanagari-Bold.ttf  
  inflating: NotoSansDevanagari-Condensed.ttf  
  inflating: NotoSansDevanagari-CondensedBlack.ttf  
  inflating: NotoSansDevanagari-CondensedBold.ttf  
  inflating: NotoSansDevanagari-CondensedExtraBold.ttf  
  inflating: NotoSansDevanagari-CondensedExtraLight.ttf  
  inflating: NotoSansDevanagari-CondensedLight.ttf  
  inflating: NotoSansDevanagari-CondensedMedium.ttf  
  inflating: NotoSansDevanagari-CondensedSemiBold.ttf  
  inflating: NotoSansDevanagari-CondensedThin.ttf  
  inflating: NotoSansDevanagari-ExtraBold.ttf  
  inflating: NotoSansDevanagari-ExtraCondensed.ttf  
  inflating: NotoSansDevanagari-ExtraCondensedBlack.ttf  
  inflating: NotoSansDevanagari-ExtraCondensedBold.ttf  
  inflating: NotoSansDevanagari-ExtraCondensedExtraBold.ttf  
  inflating: NotoSansDevanagari-ExtraCondensedExtraLight.ttf  
  

In [65]:
import matplotlib as mpl
import matplotlib.font_manager as fm
p= '/content/NotoSansDevanagari-Black.ttf'
fontprop = fm.FontProperties(fname=p, size= 15)

In [52]:
import tensorflow as tf

import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [3]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

--2021-05-08 05:53:35--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  19.3MB/s    in 19s     

2021-05-08 05:53:55 (99.2 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [4]:
!tar -xvf 'dakshina_dataset_v1.0.tar'

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.nonblock.sections.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.omit_pages.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/na

In [5]:
train_hindi_path = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_hindi_path="/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"



In [6]:
def create_dataset(path, num_examples):
  input_texts = []
  target_texts = []
  with open(path, "r", encoding="utf-8") as f:
    train_lines = f.read().split("\n")
  train_lines.pop()

#go through the train lines and split them into 3 and save input and target 
  for line in train_lines[: min(num_examples,(len(train_lines) - 1))]:
    #because we want english to devanagiri conversion 
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_text = "\t" +input_text + "\n"
    #append it to the main input texts list
    input_texts.append(input_text)
    #append it to the main target texts list 
    target_texts.append(target_text)
  
  return (input_texts,target_texts)



In [7]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [8]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  inp_lang,targ_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [28]:
# Try experimenting with the size of that dataset
num_examples = 300000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(train_hindi_path,
                                                                num_examples)
input_tensor_val,target_tensor_val,_,_=load_dataset(val_hindi_path,num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print(inp_lang.index_word[3])
print(targ_lang.index_word[1])
print(len(input_tensor), len(target_tensor), len(input_tensor_val), len(target_tensor_val))





	
44203 44203 4357 4357


In [29]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      #print(t)
      print(f'{t} ----> {lang.index_word[t]}')


In [30]:
print("Input Language; index to word mapping")
convert(inp_lang, input_tensor[3])
print()
print("Target Language; index to word mapping")
convert(targ_lang, target_tensor[3])


Input Language; index to word mapping
2 ----> 	
1 ----> a
4 ----> n
13 ----> k
15 ----> u
7 ----> r
3 ----> 


Target Language; index to word mapping
1 ----> 	
31 ----> अ
11 ----> ं
8 ----> क
24 ----> ु
4 ----> र
2 ----> 



In [31]:
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor)//BATCH_SIZE
print(steps_per_epoch)
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


690


In [37]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 22]), TensorShape([64, 21]))

In [38]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))


In [39]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)


Encoder output shape: (batch size, sequence length, units) (64, 22, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [40]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [41]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units)", attention_result.shape)
print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)


Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 22, 1)


In [42]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [43]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)


Decoder output shape: (batch_size, vocab size) (64, 65)


In [44]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [45]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['\t']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [46]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')


  print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
  print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')


Epoch 1 Batch 0 Loss 1.4134
Epoch 1 Batch 100 Loss 1.0270
Epoch 1 Batch 200 Loss 0.5469
Epoch 1 Batch 300 Loss 0.3523
Epoch 1 Batch 400 Loss 0.2417
Epoch 1 Batch 500 Loss 0.2948
Epoch 1 Batch 600 Loss 0.1667
Epoch 1 Loss 0.4662
Time taken for 1 epoch 117.98 sec

Epoch 2 Batch 0 Loss 0.2091
Epoch 2 Batch 100 Loss 0.1296
Epoch 2 Batch 200 Loss 0.1694
Epoch 2 Batch 300 Loss 0.1737
Epoch 2 Batch 400 Loss 0.1344
Epoch 2 Batch 500 Loss 0.1407
Epoch 2 Batch 600 Loss 0.1465
Epoch 2 Loss 0.1658
Time taken for 1 epoch 96.11 sec

Epoch 3 Batch 0 Loss 0.1510
Epoch 3 Batch 100 Loss 0.1169


KeyboardInterrupt: ignored

In [88]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  #sentence = '\t'+sentence+'\n'

  inputs = [inp_lang.word_index[i] for i in list(sentence)]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['\t']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] 

    if targ_lang.index_word[predicted_id] == '\n':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [66]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict,fontproperties=fontprop)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()


In [116]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  #print(attention_plot)
  #print(result)
  #print(sentence)
  #print('Input:', sentence)
  #print('Predicted translation:', result)
  return result
  #attention_plot = attention_plot[:len(list(result)),:len(list(sentence))]
  #plot_attention(attention_plot, list(sentence), list(result))


In [118]:
input,target=create_dataset(val_hindi_path, 30000)
print(input)
count=0
for i in range(len(input)):
  pred=translate(input[i])
  pred=pred.strip('\n')
  actual=target[i]
  actual=actual.strip('\n''\t')

  #print("target",actual)
  #print(list(pred))
  #print(list(actual))
  if(pred==actual):
    print(i,"correct")
    count+=1
print("Val accuracy",count/len(input))


['\tankan\n', '\tangkor\n', '\tangira\n', '\tangithi\n', '\tangrej\n', '\tangrejon\n', '\tanjaam\n', '\tanjam\n', '\tantakaran\n', '\tantkaran\n', '\tanralon\n', '\tantralon\n', '\tantarkalah\n', '\tantarklah\n', '\tantkalah\n', '\tandher\n', '\tandhera\n', '\tandheraa\n', '\tandhere\n', '\tambar\n', '\tamber\n', '\tumber\n', '\tambarnath\n', '\tahankaar\n', '\tahankar\n', '\tanhkar\n', '\takaash\n', '\takash\n', '\takeelaa\n', '\takila\n', '\tankush\n', '\taksharash\n', '\taksharashaah\n', '\taksharashah\n', '\tasharsha\n', '\takhandanand\n', '\takhbar\n', '\takhabari\n', '\takhbari\n', '\takhabaro\n', '\takhbaro\n', '\takhabaron\n', '\takhbaron\n', '\takhatar\n', '\takhtar\n', '\tagar\n', '\taugust\n', '\tagnivesh\n', '\tagrata\n', '\tagli\n', '\tugly\n', '\tachyutanand\n', '\taja\n', '\tatak\n', '\tatka\n', '\tatkav\n', '\tataki\n', '\tatki\n', '\tattention\n', '\tatarra\n', '\tatrra\n', '\tadabi\n', '\tadbi\n', '\taditi\n', '\tadhikat\n', '\tadhikaanshat\n', '\tadhikanshat\n', '\ta